### All right, this file may be a little longer than we thought, but it is neccessary.It contains all the figures of all dataframe that we have

In [1]:
import numpy as np
import pandas as pd
import csv_figures
from csv_figures import add_time_col
from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [2]:
CIHA_1400Muldoon = pd.read_excel('../../Cleaned_data/CIHA/CHIA_1400Muldoon_clean.xlsx')
CIHA_1460Muldoon = pd.read_excel('../../Cleaned_data/CIHA/CIHA_1460Muldoon_cleaned.xlsx')
CIHA_3600spenard = pd.read_excel('../../Cleaned_data/CIHA/CIHA_3600Spenard_Cleaned.xlsx') 
CIHA_7861BetnuCircle = pd.read_excel('../../Cleaned_data/CIHA/CIHA_7861BetnuCircle_cleaned.xlsx')

In [3]:
CIHA_1400Muldoon_new = add_time_col(CIHA_1400Muldoon)
CIHA_1460Muldoon_new = add_time_col(CIHA_1460Muldoon)
CIHA_3600spenard_new = add_time_col(CIHA_3600spenard)
CIHA_7861BetnuCircle_new = add_time_col(CIHA_7861BetnuCircle)

/Users/guge/Desktop/acep-solar/new/ACEP_solar/GEGU/figures/csv_figures.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])


muldoon1400_annual_values,muldoon1400_month = annual_Norm(CIHA_1400Muldoon_new)
muldoon1460_annual_values,muldoon1460_month = annual_Norm(CIHA_1460Muldoon_new)
spenard_annual_values,spenard_month = annual_Norm(CIHA_3600spenard_new)
BetnuCircle_annual_values,BetnuCircle_month = annual_Norm(CIHA_7861BetnuCircle_new)

new_1400 = pd.DataFrame({'Date':muldoon1400_month,'1400Muldoon':muldoon1400_annual_values})
new_1460 = pd.DataFrame({'Date':muldoon1460_month,'1460Muldoon':muldoon1460_annual_values})
new_spenard = pd.DataFrame({'Date':spenard_month,'Spenard':spenard_annual_values})
new_Bet = pd.DataFrame({'Date':BetnuCircle_month,'BetnuCircle':BetnuCircle_annual_values})

result = pd.merge(new_1400,new_1460,on='Date',how='outer')
result = pd.merge(result,new_spenard,on='Date',how='outer')
CIHA = pd.merge(result,new_Bet,on='Date',how='outer')
CIHA['Mean'] = CIHA.mean(axis=1)
CIHA = CIHA.sort_values(by = ['Date'])
CIHA

In [4]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 13.7, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])

In [5]:
# Get the data from the PV Watts --TMY3
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 13.7, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [6]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = CIHA_1400Muldoon_new['Date']

y_1400 = CIHA_1400Muldoon_new['Energy']
l0, = ax.plot(x ,y_1400,label='1400Muldoon')

y_TMY2 = TMY2['ac_monthly']
l1, = ax.plot(x,y_TMY2,label='TMY2')

y_TMY3 = TMY3['ac_monthly']
l2, = ax.plot(x,y_TMY3,label='TMY3')

ax.set_title("The Energy production Over one year", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("Energy/(KW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.05, 0.4, 0.1, 0.15])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# # Some 
# tick_spacing = np.linspace(0, len(fairbanks.index)-1,12, dtype = 'int')
# Date_shown = []
# for i in tick_spacing:
#     Date_shown.append(fairbanks['Date'][i])
#ax.xticks(tick_spacing,Date_shown)
#plt.annotate('THY2_high',xy=(30,1072.991757),xytext=(32,1100),arrowprops=dict(facecolor='black', shrink=0.05))
#plt.annotate('THY2_low',xy=(10,970.802066),xytext=(6,930),arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

/miniconda3/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [7]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 14.8, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 14.8, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [8]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = CIHA_1460Muldoon_new['Date']

y_1460 = CIHA_1460Muldoon_new['Energy']
l0, = ax.plot(x ,y_1460,label='1460Muldoon')

y_TMY2 = TMY2['ac_monthly']
l1, = ax.plot(x,y_TMY2,label='TMY2')

y_TMY3 = TMY3['ac_monthly']
l2, = ax.plot(x,y_TMY3,label='TMY3')

ax.set_title("The Energy production of 1460 Muldoon Over one year", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("Energy/(KW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.05, 0.4, 0.1, 0.15])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# # Some 
# tick_spacing = np.linspace(0, len(fairbanks.index)-1,12, dtype = 'int')
# Date_shown = []
# for i in tick_spacing:
#     Date_shown.append(fairbanks['Date'][i])
#ax.xticks(tick_spacing,Date_shown)
#plt.annotate('THY2_high',xy=(30,1072.991757),xytext=(32,1100),arrowprops=dict(facecolor='black', shrink=0.05))
#plt.annotate('THY2_low',xy=(10,970.802066),xytext=(6,930),arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

In [9]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18.9, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.18, "lon": -149.90, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18.9, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.18, "lon": -149.90, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [10]:
CIHA_3600spenard_new['ac_monthly_TMY2'] = TMY2['ac_monthly']
CIHA_3600spenard_new['ac_monthly_TMY3'] = TMY3['ac_monthly']
CIHA_3600spenard_new = CIHA_3600spenard_new.sort_values(by = ['Date'])

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = CIHA_3600spenard_new['Date']

y_3600 = CIHA_3600spenard_new['Energy']
l0, = ax.plot(x ,y_3600,label='3600 Spenard')

y_TMY2 = CIHA_3600spenard_new['ac_monthly_TMY2']
l1, = ax.plot(x,y_TMY2,label='TMY2')

y_TMY3 = CIHA_3600spenard_new['ac_monthly_TMY3']
l2, = ax.plot(x,y_TMY3,label='TMY3')

ax.set_title("The Energy production of 3600 Spenard Over one year", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("Energy/(KW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.05, 0.4, 0.1, 0.15])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# # Some 
# tick_spacing = np.linspace(0, len(fairbanks.index)-1,12, dtype = 'int')
# Date_shown = []
# for i in tick_spacing:
#     Date_shown.append(fairbanks['Date'][i])
#ax.xticks(tick_spacing,Date_shown)
#plt.annotate('THY2_high',xy=(30,1072.991757),xytext=(32,1100),arrowprops=dict(facecolor='black', shrink=0.05))
#plt.annotate('THY2_low',xy=(10,970.802066),xytext=(6,930),arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

In [12]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 26.9, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 26.9, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 61.20, "lon": -149.73, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [13]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = CIHA_7861BetnuCircle_new['Date']

y_7861 = CIHA_7861BetnuCircle_new['Energy']
l0, = ax.plot(x ,y_7861,label='7861 BetnuCircle')

y_mean = CIHA_7861BetnuCircle_new['Mean']
l1, = ax.plot(x,y_mean,label='Mean')

y_TMY2_low = CIHA_7861BetnuCircle_new['TMY2_low']
l2, = ax.plot(x,y_TMY2_low,'-.',label = 'TMY2_low')

y_TMY2_high = CIHA_7861BetnuCircle_new['TMY2_high']
l3, = ax.plot(x,y_TMY2_high,'-.',label = 'TMY2_high')

y_TMY3_low = CIHA_7861BetnuCircle_new['TMY3_low']
l4, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

y_TMY3_high = CIHA_7861BetnuCircle_new['TMY3_high']
l5, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')



ax.set_title("The Energy production of 7861 BetnuCircle Over one year", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("Energy/(KW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2, l3, l4, l5]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.05, 0.4, 0.1, 0.15])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# # Some 
# tick_spacing = np.linspace(0, len(fairbanks.index)-1,12, dtype = 'int')
# Date_shown = []
# for i in tick_spacing:
#     Date_shown.append(fairbanks['Date'][i])
#ax.xticks(tick_spacing,Date_shown)
#plt.annotate('THY2_high',xy=(30,1072.991757),xytext=(32,1100),arrowprops=dict(facecolor='black', shrink=0.05))
#plt.annotate('THY2_low',xy=(10,970.802066),xytext=(6,930),arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

KeyError: 'Mean'